In [16]:
import weave
import os

os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

client = weave.init("action_test_4")

Logged in as Weights & Biases user: timssweeney.
View Weave data at https://wandb.ai/timssweeney/action_test_4/weave


In [ ]:
# from weave.trace import autopatch, weave_init
# from weave.trace_server import clickhouse_trace_server_batched

# ch_server = clickhouse_trace_server_batched.ClickHouseTraceServer.from_env()
# inited_client = weave_init.InitializedClient(client)
# autopatch.autopatch()


In [20]:
from openai import OpenAI

openai_client = OpenAI()

@weave.op
def extract_name(user_input: str) -> str:
    if "Tim" in user_input:
        return "i don't know!"
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Extract the name from the user input. If there is no name, return an empty string.",
            },
            {"role": "user", "content": user_input},
        ],
        temperature=0.0,
        max_tokens=64,
        top_p=1,
    )
    return response.choices[0].message.content

calls = []
names = [
    "Tim", "Scott", "Adrian", "Jeff", "Shawn"
]
for name in names:
    res, call = extract_name.call(f"My name is {name}.")
    calls.append(call)

print(calls)

🍩 https://wandb.ai/timssweeney/action_test_4/r/call/0192b29c-96ba-7a32-8228-74a274b25a69
🍩 https://wandb.ai/timssweeney/action_test_4/r/call/0192b29c-96bd-78f2-8966-473c36799e10
🍩 https://wandb.ai/timssweeney/action_test_4/r/call/0192b29c-9827-71f0-9997-199922a0b5a5
🍩 https://wandb.ai/timssweeney/action_test_4/r/call/0192b29c-999b-7762-93a5-f70678bba80b
🍩 https://wandb.ai/timssweeney/action_test_4/r/call/0192b29c-9ae2-70c1-a03b-ee5f915e48fe
[Call(_op_name=<Future at 0x2b7dc7cd0 state=finished returned str>, trace_id='0192b29c-96ba-7a32-8228-74b76ac97678', project_id='timssweeney/action_test_4', parent_id=None, inputs={'user_input': 'My name is Tim.'}, id='0192b29c-96ba-7a32-8228-74a274b25a69', output="i don't know!", exception=None, summary={}, display_name=None, attributes=AttributesDict({'weave': {'client_version': '0.51.18-dev0', 'source': 'python-sdk', 'os_name': 'Darwin', 'os_version': 'Darwin Kernel Version 23.2.0: Wed Nov 15 21:53:18 PST 2023; root:xnu-10002.61.3~2/RELEASE_ARM64

In [21]:
from weave.flow import action_objects
from weave.trace.weave_client import get_ref
from weave.trace_server.interface import actions

action = action_objects.ActionWithConfig(
    name="is_name_extracted",
    action=actions.BuiltinAction(
        name="openai_completion",
    ),
    config={
        "model": "gpt-4o-mini",
        "system_prompt": "Given the following prompt and response, determine if the name was extracted correctly.",
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "is_name_extracted",
                "schema": {
                    "type": "object",
                    "properties": {"is_extracted": {"type": "boolean"}},
                    "required": ["is_extracted"],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        },
    },
)
mapping = action_objects.ActionOpMapping(
    action=action,
    op_name=get_ref(extract_name).name,
    op_digest=get_ref(extract_name).digest,
    input_mapping={
        "prompt": "inputs.user_input",
        "response": "output",
    },
)

In [22]:

from weave.trace_server import trace_server_interface as tsi

res = client.server.execute_batch_action(req=tsi.ExecuteBatchActionReq(
    project_id=client._project_id(), call_ids=[c.id for c in calls], mapping=mapping
))